In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import keras
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential

In [4]:
data_dir = r'C:\Users\iceki\OneDrive\Desktop\SSW 345\ISE-490---Machine-Learning\Final Project\Train'

In [5]:
categories = os.listdir(data_dir)

In [6]:
IMG_SIZE = 32
CHANNELS = 3

In [8]:
def preprocess_data():
    data = []
    for category in categories:
        path = os.path.join(data_dir, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                data.append([img_array, class_num])
            except Exception as e:
                print(f"Error processing image: {e}")
    return data

In [9]:
data = preprocess_data()

In [11]:
np.random.shuffle(data)

[[array([[[74, 76, 76],
          [75, 77, 77],
          [74, 76, 76],
          ...,
          [81, 83, 83],
          [80, 83, 81],
          [79, 82, 80]],
  
         [[74, 76, 76],
          [75, 77, 77],
          [74, 76, 76],
          ...,
          [80, 82, 82],
          [81, 84, 82],
          [80, 83, 81]],
  
         [[74, 76, 76],
          [75, 77, 77],
          [75, 77, 77],
          ...,
          [82, 84, 84],
          [81, 83, 83],
          [83, 85, 85]],
  
         ...,
  
         [[52, 54, 54],
          [62, 64, 64],
          [53, 55, 55],
          ...,
          [74, 76, 76],
          [69, 71, 71],
          [72, 74, 74]],
  
         [[58, 60, 60],
          [60, 62, 62],
          [60, 62, 62],
          ...,
          [66, 68, 68],
          [59, 61, 61],
          [59, 61, 61]],
  
         [[57, 59, 59],
          [56, 58, 58],
          [59, 61, 61],
          ...,
          [66, 68, 68],
          [64, 67, 67],
          [67, 69, 69]]], dtype=u

In [16]:
X = np.array([entry[0] for entry in data])
y = np.array([entry[1] for entry in data])

In [17]:
X = X / 255.0

In [18]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [41]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, CHANNELS)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(200, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(len(categories), activation='softmax'))


In [42]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [43]:
model.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test))

Epoch 1/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.1433 - loss: 2.0823 - val_accuracy: 0.1730 - val_loss: 2.0605
Epoch 2/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.1985 - loss: 2.0341 - val_accuracy: 0.2187 - val_loss: 2.0149
Epoch 3/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.2977 - loss: 1.8983 - val_accuracy: 0.3658 - val_loss: 1.7575
Epoch 4/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3837 - loss: 1.6481 - val_accuracy: 0.5010 - val_loss: 1.4667
Epoch 5/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5069 - loss: 1.3855 - val_accuracy: 0.5348 - val_loss: 1.3471
Epoch 6/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6134 - loss: 1.1491 - val_accuracy: 0.5706 - val_loss: 1.2990
Epoch 7/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6613 - loss: 0.9795 - val_accuracy: 0.6183 - val_loss: 1.2060
Epoch 8/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7416 - loss: 0.7664 - val_accuracy: 0.5924 - val_loss

In [53]:
_, test_acc = model.evaluate(x_test,y_test)
print(test_acc)

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6532 - loss: 1.9215
0.6202783584594727


In [44]:
def preprocess_image(image_path):
    img_array = cv2.imread(image_path)
    img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    img_array = img_array / 255.0
    return img_array.reshape(-1, IMG_SIZE, IMG_SIZE, CHANNELS)

In [45]:
def classify_image(image_path):
    preprocessed_image = preprocess_image(image_path)
    prediction = model.predict(preprocessed_image)
    predicted_class_index = np.argmax(prediction)
    predicted_class = categories[predicted_class_index]
    return predicted_class

In [58]:
test_image_path = r'C:\Users\iceki\OneDrive\Desktop\SSW 345\ISE-490---Machine-Learning\Final Project/Toyota.jpg'

In [59]:
predicted_class = classify_image(test_image_path)
print("Predicted class:", predicted_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted class: toyota
